In [26]:
import numpy as np
import pandas as pd

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns

#time
from datetime import datetime
from datetime import timedelta

# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

In [27]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [28]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [29]:
# 選那家股票
company_name = '國巨'
#文章包含那家字
company_words = '國巨'

In [30]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36


In [31]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,0.0
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,0.0


In [32]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [33]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,0.0,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,0.0,--,68.62


In [34]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,--,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,--,68.62


In [35]:
df_trend['tag']='even'
df_trend.loc[df_trend['fluctuation'] >=  0.03, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -0.03, 'tag'] = 'down'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,1128.0,59378.0,697,651594.0,9.38,1.37,NaN,even,NaN
1,2327 國巨,2016-01-05,68.10,68.49,67.33,68.36,1067.0,55785.0,703,651594.0,9.34,1.36,-0.003789,even,68.62


In [36]:
up_trend=df_trend[df_trend['tag']=='up']
len(up_trend)

114

In [37]:
down_trend=df_trend[df_trend['tag']=='down']
len(down_trend)

89

In [95]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
df_company.head(2)

6376


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07
2458,comjj45,0,◆買超 代碼 股票名稱 買 超 買 進 賣 出 買 進 賣 出 ...,1452872063254_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1452871811.A.2FB...,2016-01-15 23:30:09,Stock,Ptt,[其他] 1/15 台灣集中市場自營商買賣超前30名,2016-01-15


In [96]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [97]:
# 'content'全部切詞
corpus = []  # array
for index, row in df_company.iterrows():  
    not_cut = df_company.loc[index,'content']
#     not_cut = row['description']                    # 跟上一行一樣意思
    seg_generator = jieba.cut(not_cut, cut_all=False)  # genarator
    seglist = list(seg_generator)                     # 整篇文章string切出來的list
    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟
    corpus.append(' '.join(seglist))          # ' '.join(seg_generator)也可

df_company["content2"]=corpus  

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [98]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07,新台幣 連八貶 創 80 新低 昨收 33.453 元 農曆 封關 前 貶破 34 元 人行...
2458,comjj45,0,◆買超 代碼 股票名稱 買 超 買 進 賣 出 買 進 賣 出 ...,1452872063254_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1452871811.A.2FB...,2016-01-15 23:30:09,Stock,Ptt,[其他] 1/15 台灣集中市場自營商買賣超前30名,2016-01-15,買超 代碼 股票 名稱 買 超 買 進 賣 買 進 賣 張 數 張 數 張 數 價 價 00...


In [99]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [100]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [101]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [102]:
d.days #只取天數

1

In [103]:
int(d.days) #幾天 轉整數

1

In [104]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [105]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
865,財訊快報,0,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,1452126731305_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 07:58:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-07,新台幣 連八貶 創 80 新低 昨收 33.453 元 農曆 封關 前 貶破 34 元 人行...


In [106]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [107]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [108]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label'] = df_trend.loc[index,'tag']
    except:
        continue

In [109]:
print(len(df_company[df_company['label']=='down']))
df_company[df_company['label']=='down'].head()

1637


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label
17050,Moneydj理財網,0,MoneyDJ新聞 2016-04-21 13:02:47 記者 蔡承啟 報導<BR>全球晶...,1461216622247_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-04-21 13:02:00,重大要聞,yahoo股市,都是日圓的錯！晶片電阻廠KOA營益降3成、本季恐更慘,2016-04-21,MoneyDJ 新聞 2016 04 21 13 02 47 記者 蔡承啟 報導 全球 晶片...,down
17072,鉅亨網,0,國安基金宣布退場，並宣示賣紅不賣黑，引發台股上漲恐有壓力，不過台股在電子權值股台積電及鴻海領...,1461222022467_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-04-21 14:45:00,重大要聞,yahoo股市,台股盤後－國安基金賣股沒在怕 台積電鴻海領軍止穩 上漲54點,2016-04-21,國安 基金 宣布 退場 宣示 賣紅不賣 黑 引發 台股 上漲 恐有 壓力 台股 電子 權值股...,down
23645,Yahoo奇摩股市,0,（開盤日9:00出刊）美股道瓊收復盤中120點跌幅，收盤上漲2點；財長許虞哲今赴立院報告，擬...,1464828254666_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-06-02 08:27:00,重大要聞,yahoo股市,【Y早報】財長：檢討富人稅，列稅改優先,2016-06-02,開盤 00 出刊 美股 道瓊 收復 盤中 120 點 跌幅 收盤 上漲 點 財長 許虞哲 今...,down
60743,kala0620,4,十年前國巨併購大毅鬧得沸沸揚揚<BR>短短幾個月股價炒到兩百多<BR>十年後的今日<BR>收...,1486073899471_F01,forum,http://www.mobile01.com/topicdetail.php?f=291&...,2017-02-03 06:02:00,投資與理財(生活娛樂),mobile01,大毅收購案各位怎麼看,2017-02-03,十年 前 國巨 併購 大毅 鬧 沸沸 揚揚 短短 幾個 股價 炒到 兩百多 十年 今日 收購...,down
65014,Moneydj理財網,0,1.美股三大指數週二下跌，週三早盤台股以上漲0.65點的9751.12點開出後，指數便在平盤...,1488358695643_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2017-03-01 16:29:00,重大要聞,yahoo股市,日盛投顧：台股恐向9550點~月線尋求支撐,2017-03-01,美股 三大 指數 週二 下跌 週三 早盤 台股 上漲 0.65 點的 9751.12 點 開...,down


In [110]:
print(len(df_company[df_company['label']=='up']))
df_company[df_company['label']=='up'].head()

1577


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label
58036,鉅亨網,0,鉅亨台北資料中心◆盤勢分析<BR>1. 週二美股漲跌互見，其中 Nasdaq 走揚再創歷史新...,1484122174687_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2017-01-11 15:35:00,重大要聞,yahoo股市,日盛投顧盤後－外資持續買超、台股高檔震盪,2017-01-11,鉅 亨 台北 資料 中心 盤勢 分析 週二 美股 漲跌 互見 Nasdaq 走揚 再創 歷史...,up
58056,Moneydj理財網,0,1.週二美股漲跌互見，其中Nasdaq走揚再創歷史新高，週三亞股漲跌互見，台股早盤則以上漲2...,1484126667810_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2017-01-11 17:02:00,重大要聞,yahoo股市,日盛投顧：台股5日線未走平下彎，仍有高點可期,2017-01-11,週二 美股 漲跌 互見 Nasdaq 走揚 再創 歷史 新高 週三 亞股 漲跌 互見 台股 ...,up
58091,primotore,9,昨收62塊，今日強勁的買盤將股價推升至63.9 國巨算是價值被低估的yield play...,1484145758442_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1484144521.A.781...,2017-01-11 22:21:59,Stock,Ptt,Re: [標的] 2327 國巨,2017-01-11,昨收 62 塊 今日 強勁 買盤 股價 推升 63.9 國巨 算是 價值 低估 yield ...,up
59316,財訊快報,0,美國白宮的新主人川普將於20日就任美國總統，由於川普是政治素人，且選舉當時不乏驚人之語，目前...,1484806292545_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2017-01-19 13:47:00,重大要聞,yahoo股市,盤後分析：市場靜待川普就職，台股狹幅震盪，被動元件人氣旺,2017-01-19,美國 白宮 新 主人 川普 將於 20 就任 美國 總統 川普 政治 素人 選舉 當時 不乏...,up
59356,Moneydj理財網,0,1.周四台股仍呈現整理型態，2330台積電持續整理，大立光漲多回檔，金融股持續回檔，終場指數...,1484817994503_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2017-01-19 17:02:00,重大要聞,yahoo股市,日盛投顧：台股站回9350點上方，才有轉強機會,2017-01-19,周四 台股 呈現 整理 型態 2330 台積電 持續 整理 大立光 漲多 回檔 金融股 持續...,up
